In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Set image size & batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define dataset directory paths
dataset_dir = r"D:\01 STUDY MATERIAL\ai project\Breast-Splitted"
train_dir = f"{dataset_dir}/train"
val_dir = f"{dataset_dir}/val"
test_dir = f"{dataset_dir}/test"

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# Validation & Test: No augmentation, just rescaling
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load dataset using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary", shuffle=False
)

# Load Pretrained ResNet-50 Model (without classification head)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
base_model.trainable = False

# Add a new classification head
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)  # Helps prevent overfitting
x = Dense(1, activation="sigmoid")(x)  # Binary classification output

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train only the new classification head first
model.fit(train_generator, validation_data=val_generator, epochs=5)

# Unfreeze some layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Keep most layers frozen
    layer.trainable = False

# Re-compile with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
model.fit(train_generator, validation_data=val_generator, epochs=3)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the trained model
# model.save("resnet50_transfer_learning_tf.h5")

Found 4745 images belonging to 2 classes.
Found 1581 images belonging to 2 classes.
Found 1583 images belonging to 2 classes.
Epoch 1/5
149/149 [==============================] - 236s 2s/step - loss: 0.7054 - accuracy: 0.6662 - val_loss: 0.6126 - val_accuracy: 0.6863
Epoch 2/5
149/149 [==============================] - 204s 1s/step - loss: 0.6246 - accuracy: 0.6864 - val_loss: 0.6235 - val_accuracy: 0.6863
Epoch 3/5
149/149 [==============================] - 224s 2s/step - loss: 0.6229 - accuracy: 0.6866 - val_loss: 0.6117 - val_accuracy: 0.6863
Epoch 4/5
149/149 [==============================] - 209s 1s/step - loss: 0.6276 - accuracy: 0.6864 - val_loss: 0.6154 - val_accuracy: 0.6863
Epoch 5/5
149/149 [==============================] - 207s 1s/step - loss: 0.6229 - accuracy: 0.6864 - val_loss: 0.6091 - val_accuracy: 0.6863
Epoch 1/3
149/149 [==============================] - 217s 1s/step - loss: 0.6219 - accuracy: 0.6891 - val_loss: 0.6111 - val_accuracy: 0.6869
Epoch 2/3
149/149 [===

In [2]:
model.save("resnet50_transfer_learning_tf.h5")